# 3 - Organizing Pandas: Summary

In [75]:
import pandas as pd
import numpy as np
import pandas_datareader as web
import datetime as dt
import scipy as sp
import math as m

start = dt.datetime(2012,1,1)
end = dt.datetime(2012,12,30)

msft = web.DataReader("WIKI/MSFT", "quandl", start, end)
aapl = web.DataReader("WIKI/AAPL", "quandl", start, end)



# Concatenation

## Adding Dataframes Together

In [32]:
"""Problem"""

msftAV = msft[["AdjClose", "Volume"]] #by columns [[]], by index and column [][[]]
aaplAV = aapl[["AdjClose", "Volume"]]
(pd.concat([msftAV, aaplAV]))[:5]

,AdjClose,Volume
Date,,
2012-12-28,23.257183,28239900.0
2012-12-27,23.616333,39394000.0
2012-12-26,23.528736,31631100.0
2012-12-24,23.703931,20842400.0
2012-12-21,24.045562,98776500.0


In [9]:
"""Solution"""
pd.concat([msftAV[:5], aaplAV[:5]], axis = 1,keys = ["MSFT", "AAPL"], join = "outer")

MSFT                   AAPL            
             AdjClose      Volume   AdjClose      Volume
Date                                                    
2012-12-28  23.257183  28239900.0  66.081242  12652800.0
2012-12-27  23.616333  39394000.0  66.790671  16254300.0
2012-12-26  23.528736  31631100.0  66.523474  10801300.0
2012-12-24  23.703931  20842400.0  67.453053   6276900.0
2012-12-21  24.045562  98776500.0  67.344385  21295300.0

# Merge by Value

In [12]:
"""Reset index is needed so as to completely expose the columns as values"""
msftAR = msft[["AdjClose"]].reset_index()
msftVR = msft[["Volume"]].reset_index() 
(pd.merge(msftAR, msftAV, how = "outer"))[:5] #MERGE ==> NO []

,Date,AdjClose,Volume
0,2012-12-28,23.257183,28239900.0
1,2012-12-27,23.616333,39394000.0
2,2012-12-26,23.528736,31631100.0
3,2012-12-24,23.703931,20842400.0
4,2012-12-21,24.045562,98776500.0


# Pivoting

In [20]:
"""Transpose tables and insert values"""
#msft.insert(0, "Symbol", "MSFT")
#aapl.insert(0, "Symbol", "AAPL")
combined = pd.concat([msft, aapl]).sort_index()

s4p = combined.reset_index()
s4p[:5]

,Date,Symbol,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
0,2012-01-03,MSFT,26.5500,26.960,26.39,26.765,64731500.0,0.0,1.0,22.609162,22.958305,22.472910,22.792249,64731500.0
1,2012-01-03,AAPL,409.4000,412.500,409.00,411.230,10793600.0,0.0,1.0,52.613606,53.011999,52.562200,52.848787,75555200.0
2,2012-01-04,AAPL,410.0000,414.680,409.28,413.440,9286500.0,0.0,1.0,52.690714,53.292160,52.598184,53.132802,65005500.0
3,2012-01-04,MSFT,26.8199,27.470,26.78,27.400,80516100.0,0.0,1.0,22.839000,23.392605,22.805022,23.332995,80516100.0
4,2012-01-05,MSFT,27.3800,27.728,27.29,27.680,56081400.0,0.0,1.0,23.315964,23.612310,23.239323,23.571435,56081400.0


In [46]:
closes = s4p.pivot(index = "Date", columns = "Symbol", values = "AdjClose")
closes[:3]

Symbol,AAPL,MSFT
Date,,
2012-01-03,52.848787,22.792249
2012-01-04,53.132802,23.332995
2012-01-05,53.722681,23.571435


# Stack/Unstack

In [29]:
"""Un-pivot"""
stack = closes.stack()
stack[:5]

Date        Symbol
2012-01-03  AAPL      52.848787
            MSFT      22.792249
2012-01-04  AAPL      53.132802
            MSFT      23.332995
2012-01-05  AAPL      53.722681
dtype: float64

In [31]:
"""Re-pivot"""
(stack.unstack())[:3]

Symbol,AAPL,MSFT
Date,,
2012-01-03,52.848787,22.792249
2012-01-04,53.132802,23.332995
2012-01-05,53.722681,23.571435


# Melting

In [48]:
"""unique-id variables"""
#Creates unique id variable Ffor everything --> melts all other data on date and symbol (like a tag)
melted = pd.melt(s4p, id_vars = ["Date", "Symbol"])

#Filter by tags
melted[(melted.Date == "2012-01-03") & (melted.Symbol == "MSFT")]


,Date,Symbol,variable,value
0,2012-01-03,MSFT,Open,2.655000e+01
498,2012-01-03,MSFT,High,2.696000e+01
996,2012-01-03,MSFT,Low,2.639000e+01
1494,2012-01-03,MSFT,Close,2.676500e+01
1992,2012-01-03,MSFT,Volume,6.473150e+07
2490,2012-01-03,MSFT,ExDividend,0.000000e+00
2988,2012-01-03,MSFT,SplitRatio,1.000000e+00
3486,2012-01-03,MSFT,AdjOpen,2.260916e+01
3984,2012-01-03,MSFT,AdjHigh,2.295830e+01
4482,2012-01-03,MSFT,AdjLow,2.247291e+01


# Split-apply-combine

In [90]:
"""1) Split into groups"""
#Reset the index on the pivot df and additional date columns for grouping
s4g = combined[["Symbol", "AdjClose"]].reset_index()
s4g.insert(1, "Year", pd.DatetimeIndex(s4g["Date"]).year) #Get year from date as index
s4g.insert(2, "Month", pd.DatetimeIndex(s4g["Date"]).month) #Get month from date as index

#To use the groupby method:
def print_groups(obj):
    for name, group in obj:
        print (name)
        print (group.head())
        
#Single grouping
grouped = s4g.groupby("Symbol") #in this way you can grouped.getgroup("symbol you want to find")
#Multi-level grouping
mcg = s4g.groupby(["Symbol", "Year", "Month"])
#Hierarchical grouping: Set_index to give each group index a hierarchy (level: 0, 1, 2 ...)
mi = s4g.set_index(["Symbol", "Year", "Month"]) #groupby(level = 0)
#Multi-level Hierarchical grouping:
mih = mi.groupby(level = ["Symbol", "Year", "Month"])
print_groups(mih)

('AAPL', 2012, 1)
                        Date   AdjClose
Symbol Year Month                      
AAPL   2012 1     2012-01-03  52.848787
            1     2012-01-04  53.132802
            1     2012-01-05  53.722681
            1     2012-01-06  54.284287
            1     2012-01-09  54.198183
('AAPL', 2012, 2)
                        Date   AdjClose
Symbol Year Month                      
AAPL   2012 2     2012-02-01  58.626773
            2     2012-02-02  58.489263
            2     2012-02-03  59.075287
            2     2012-02-06  59.626612
            2     2012-02-07  60.251189
('AAPL', 2012, 3)
                        Date   AdjClose
Symbol Year Month                      
AAPL   2012 3     2012-03-01  69.971984
            3     2012-03-02  70.063229
            3     2012-03-05  68.518491
            3     2012-03-06  68.145801
            3     2012-03-07  68.201062
('AAPL', 2012, 4)
                        Date   AdjClose
Symbol Year Month                      
AAPL   2

In [91]:
"""2) Apply"""
#apply a function to all groups
mih.agg([np.mean, np.std, np.min, np.max])


AdjClose                                
                        mean       std       amin       amax
Symbol Year Month                                           
AAPL   2012 1      55.078240  1.714656  52.848787  58.664042
            2      63.944808  3.404731  58.489263  69.711100
            3      74.217792  3.969773  68.145801  79.372778
            4      77.879832  2.920890  72.003789  81.764423
            5      72.568345  1.899324  68.127809  75.306597
            6      73.839219  0.966035  72.095034  75.490372
            7      77.245628  1.426440  73.880092  78.948682
            8      82.859921  3.154259  77.983543  87.205008
            9      87.965057  1.840911  85.257453  90.614841
            10     81.917888  2.736182  76.833538  86.659073
            11     73.113511  2.842178  68.160044  76.990982
            12     68.993735  2.654920  66.081242  76.014490
MSFT   2012 1      24.283134  0.778694  22.792249  25.317152
            2      26.345852  0.606879  25.453403  27.318944
            3      27.596755  0.291774  27.048927  28.159000
            4      27.005638  0.539541  26.011715  27.790404
            5      26.001320  0.729976  25.075128  27.438952
            6      25.609699  0.637932  24.548774  26.688703
            7      25.675853  0.540479  24.704092  26.542014
            8      26.301014  0.441266  25.187301  26.839802
            9      26.736347  0.402019  25.849595  27.317533
            10     25.246699  0.622257  24.216624  26.084118
            11     24.279917  0.931986  23.231692  25.936456
            12     23.632673  0.340400  23.099507  24.247037